In [ ]:


# This code was used to aggregate number of pixels coresponding to the IFL_2000 global forest dataset.

# """ Created on Mon July 2 2024

# @author: Pius N.Nwachukwu """


In [ ]:
pip install earthengine-api matplotlib geopandas folium


In [ ]:
# Import necessary libraries
import ee
import geopy
import folium
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import auth
from sklearn.metrics.pairwise import rbf_kernel
import numpy as np
from google.colab import drive
import gdown
from shapely.geometry import Polygon
import geopandas as gpd


In [ ]:
# Initialize Earth Engine
auth.authenticate_user()
ee.Initialize(project='ee-')

In [ ]:

# Function to aggregate pixel count
def aggregate_pixel_count(product, geometries, start_date, end_date):
    # Load MODIS ImageCollection
    collection = ee.ImageCollection(product)\
        .filterDate(start_date, end_date)

    # Calculate pixel count for each geometry
    count_images = []
    for geometry in geometries:
        count_image = collection.filterBounds(geometry) \
            .reduce(ee.Reducer.count()) \
            .select([0], ['pixel_count']) \
            .clip(geometry)
        count_images.append(count_image)

    return count_images

# Example usage:
product = 'MODIS/061/MOD13A2'  # MODIS product path
# Define rectangles for different continents
continents = [
    ee.Geometry.Rectangle([19.85,-3.16, 22.39,-1.31]),  # Africa
    ee.Geometry.Rectangle([101.23,29.02, 102.10,29.54]),    # Asia
    ee.Geometry.Rectangle([122.37,-33.08, 122.85,-32.49]),   # Aust_Ocean
    ee.Geometry.Rectangle([22.45,45.12, 22.97,45.47]),  # Europe
    ee.Geometry.Rectangle([-121.68,48.02, -119.80,49.01]),    # Nth_America
    ee.Geometry.Rectangle([-60.70,-9.18, -58.42,-7.56]),   # Sth_America
]
start_date = '2020-01-01'
end_date = '2020-12-30'

pixel_count_images = aggregate_pixel_count(product, continents, start_date, end_date)

# Define a function to add Earth Engine layers to a folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map
map_center = [20, 20]
my_map = folium.Map(location=map_center, zoom_start=3)

# Visualization parameters
vis_params = {
    'min': 0,
    'max': 10,
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
}

# Add the pixel count layers to the map
for i, count_image in enumerate(pixel_count_images):
    my_map.add_ee_layer(count_image, vis_params, f'Pixel Count {i+1}')

# Add a layer control panel to the map
my_map.add_child(folium.LayerControl())

# Display the map
my_map  # Or my_map.save("pixel_count_map.html") to save it

# Or this

In [ ]:

# Function to aggregate pixel count
def aggregate_pixel_count(product, geometries, start_date, end_date):
    # Load MODIS ImageCollection
    collection = ee.ImageCollection(product)\
        .filterDate(start_date, end_date)

    # Calculate pixel count for each geometry
    count_images = []
    for geometry in geometries:
        count_image = collection.filterBounds(geometry) \
            .reduce(ee.Reducer.count()) \
            .select([0], ['pixel_count']) \
            .clip(geometry)
        count_images.append(count_image)

    return count_images

# Example usage:
product = 'MODIS/061/MOD13A2'  # MODIS product path
# Define rectangles for different continents
continents = [
    ee.Geometry.Rectangle([19.85,-3.16, 22.39,-1.31]),  # Africa
    ee.Geometry.Rectangle([101.23,29.02, 102.10,29.54]),    # Asia
    ee.Geometry.Rectangle([122.37,-33.08, 122.85,-32.49]),   # Aust_Ocean
    ee.Geometry.Rectangle([22.45,45.12, 22.97,45.47]),  # Europe
    ee.Geometry.Rectangle([-121.68,48.02, -119.80,49.01]),    # Nth_America
    ee.Geometry.Rectangle([-60.70,-9.18, -58.42,-7.56]),   # Sth_America

]
start_date = '2020-02-01'
end_date = '2020-04-30'

pixel_count_images = aggregate_pixel_count(product, continents, start_date, end_date)

# Define a function to add Earth Engine layers to a folium map
def add_ee_layer(self, ee_image_object, vis_params, name, label): # Added label parameter
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True,
        show=False # Initially hide the layers
    ).add_to(self)

    # Add a label for the layer
    folium.Marker(
        location=ee_image_object.geometry().centroid().coordinates().getInfo()[::-1], # Get the center of the geometry
        popup=label,
        icon=folium.Icon(color='blue')
    ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map
map_center = [20, 20]  # Adjust for a global view
my_map = folium.Map(location=map_center, zoom_start=3)

# Visualization parameters
vis_params = {
    'min': 0,
    'max': 10,  # Adjust this based on expected range of pixel counts
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
}

# Labels for each continent
continent_labels = [
    'Africa', 'Asia', 'Australia/Oceania', 'Europe', 'North America', 'South America'
]

# Add the pixel count layers to the map with labels
for i, count_image in enumerate(pixel_count_images):
    my_map.add_ee_layer(count_image, vis_params, f'Pixel Count {i+1}', continent_labels[i]) # Pass the label from continent_labels

# Add a layer control panel to the map
my_map.add_child(folium.LayerControl())

# Display the map
my_map  # Or my_map.save("pixel_count_map.html") to save it

In [ ]:
# Use this to print the pixel count

def aggregate_pixel_count(product, geometry, start_date, end_date):
    collection = ee.ImageCollection(product) \
        .filterBounds(geometry) \
        .filterDate(start_date, end_date)

    count_image = collection.reduce(ee.Reducer.count()).select([0], ['pixel_count'])
    count_image = count_image.clip(geometry)
    return count_image

# Example usage:
product = 'MODIS/061/MOD13A2'
start_date = '2000-02-01'
end_date = '2024-04-30'

# Define geometries for each continent
geometries = {
    'Africa': ee.Geometry.Rectangle([19.85,-3.16, 22.39,-1.31]),
    'Asia': ee.Geometry.Rectangle([101.23,29.02, 102.10,29.54]),
    'Aust_Ocean': ee.Geometry.Rectangle([122.37,-33.08, 122.85,-32.49]),
    'Europe': ee.Geometry.Rectangle([22.45,45.12, 22.97,45.47]),
    'Nth_America': ee.Geometry.Rectangle([-121.68,48.02, -119.80,49.01]),
    'Sth_America': ee.Geometry.Rectangle([-60.70,-9.18, -58.42,-7.56]),
}

pixel_counts = {}
for continent, geometry in geometries.items():
    pixel_count_image = aggregate_pixel_count(product, geometry, start_date, end_date)
    # Get the total pixel count for the continent
    pixel_count = pixel_count_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geometry,
        scale=500,
        maxPixels=1e13  # Increase the maximum number of pixels allowed
        #bestEffort=True
    ).get('pixel_count').getInfo()
    pixel_counts[continent] = pixel_count

# Print the pixel counts for each continent
for continent, count in pixel_counts.items():
    print(f'Pixel count for {continent}: {count}')

Pixel count for Africa: 129715142.91764683
Pixel count for Asia: 12479566.815686226
Pixel count for Aust_Ocean: 7814286.243137279
Pixel count for Europe: 5020308.470588274
Pixel count for Nth_America: 51338842.070588194
Pixel count for Sth_America: 101914020.35686333


In [ ]:
pip install earthengine-api matplotlib geopandas folium


In [ ]:
# Import necessary libraries
import ee
import geopy
import folium
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import auth
from sklearn.metrics.pairwise import rbf_kernel
import numpy as np
from google.colab import drive
import gdown
# Import the Polygon class from shapely.geometry
from shapely.geometry import Polygon
import geopandas as gpd


In [ ]:
# Initialize Earth Engine
auth.authenticate_user()
ee.Initialize(project='ee-pnnwachukwu17')

 # For single continent like Europe

In [ ]:
# For single continent like Europe

# Initialize Earth Engine
# auth.authenticate_user()  # Assuming you have already authenticated
ee.Initialize(project='ee-pnnwachukwu17')

# Function to aggregate pixel count
def aggregate_pixel_count(product, geometry, start_date, end_date):
    # Load MODIS ImageCollection
    collection = ee.ImageCollection(product) \
        .filterBounds(geometry) \
        .filterDate(start_date, end_date)

    # Reduce the collection to get the total pixel count in the region of interest
    count_image = collection.reduce(ee.Reducer.count()).select([0], ['pixel_count']) # Select the first band and rename it

    # Clip to the geometry
    count_image = count_image.clip(geometry)

    return count_image

# Example usage:
product = 'MODIS/061/MOD13A2'  # MODIS product path
# Define a square geometry (replace with your desired coordinates)
geometry = ee.Geometry.Rectangle([
    [-122.1, 37.4],  # Bottom left corner
    [-122.0, 37.5]   # Top right corner
])
start_date = '2020-01-01'
end_date = '2020-12-31'

pixel_count_image = aggregate_pixel_count(product, geometry, start_date, end_date)

# Define a function to add Earth Engine layers to a folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map
map_center = geometry.centroid().coordinates().getInfo()[::-1]
my_map = folium.Map(location=map_center, zoom_start=12)

# Visualization parameters
vis_params = {
    'min': 0,
    'max': 10,  # Adjust this based on expected range of pixel counts
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
}

# Add the pixel count layer to the map
my_map.add_ee_layer(pixel_count_image, vis_params, 'Pixel Count')

# Add a layer control panel to the map
my_map.add_child(folium.LayerControl())

# Display the map
my_map  # Or my_map.save("pixel_count_map.html") to save it



Total number of pixels: None


In [ ]:
# This is used to print the number of pixels

# Function to aggregate pixel count
def aggregate_pixel_count(product, geometry, start_date, end_date):
    # Load MODIS ImageCollection
    collection = ee.ImageCollection(product) \
        .filterBounds(geometry) \
        .filterDate(start_date, end_date)

    # Reduce the collection to get the total pixel count in the region of interest
    count_image = collection.reduce(ee.Reducer.count()).select([0], ['pixel_count'])

    # Clip to the geometry
    count_image = count_image.clip(geometry)

    return count_image

# Example usage:
product = 'MODIS/061/MOD13A2'  # MODIS product path

# Define a square geometry (replace with your desired coordinates)
geometry = ee.Geometry.Rectangle([
    22.45,45.12,  # Lower left corner (longitude, latitude)
    22.97,45.47   # Upper right corner (longitude, latitude)
])

start_date = '2020-01-01'
end_date = '2020-12-31'

pixel_count_image = aggregate_pixel_count(product, geometry, start_date, end_date)

# Get the total number of pixels
pixel_count = pixel_count_image.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=geometry,
    scale=500  # Adjust scale as needed
).get('pixel_count').getInfo()

print("Total number of pixels:", pixel_count)

# ... (rest of the code for creating and displaying the map)

Total number of pixels: 207374.13333333423


 # For Multiple Continents

In [ ]:
import ee
import folium
import matplotlib.pyplot as plt

# Initialize Earth Engine
# auth.authenticate_user()  # Assuming you have already authenticated
ee.Initialize(project='ee-pnnwachukwu17')

# Function to aggregate pixel count
def aggregate_pixel_count(product, geometries, start_date, end_date):
    # Load MODIS ImageCollection
    collection = ee.ImageCollection(product)\
        .filterDate(start_date, end_date)

    # Calculate pixel count for each geometry
    count_images = []
    for geometry in geometries:
        count_image = collection.filterBounds(geometry) \
            .reduce(ee.Reducer.count()) \
            .select([0], ['pixel_count']) \
            .clip(geometry)
        count_images.append(count_image)

    return count_images

# Example usage:
product = 'MODIS/061/MOD13A2'  # MODIS product path
# Define rectangles for different continents (replace with your desired coordinates)
continents = [
    ee.Geometry.Rectangle([19.85,-3.16, 22.39,-1.31]),  # Africa (approx.)
    ee.Geometry.Rectangle([101.23,29.02, 102.10,29.54]),    # Asia (approx.)
    ee.Geometry.Rectangle([122.37,-33.08, 122.85,-32.49]),   # Aust_Ocean (approx.)
    ee.Geometry.Rectangle([22.45,45.12, 22.97,45.47]),  # Europe (approx.)
    ee.Geometry.Rectangle([-121.68,48.02, -119.80,49.01]),    # Nth_America (approx.)
    ee.Geometry.Rectangle([-60.70,-9.18, -58.42,-7.56]),   # Sth_America (approx.)
    # Add more rectangles for other continents
]
start_date = '2020-01-01'
end_date = '2020-12-30'

pixel_count_images = aggregate_pixel_count(product, continents, start_date, end_date)

# Define a function to add Earth Engine layers to a folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map (adjust map center as needed)
map_center = [20, 20]  # Adjust for a global view
my_map = folium.Map(location=map_center, zoom_start=3)

# Visualization parameters
vis_params = {
    'min': 0,
    'max': 10,  # Adjust this based on expected range of pixel counts
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
}

# Add the pixel count layers to the map
for i, count_image in enumerate(pixel_count_images):
    my_map.add_ee_layer(count_image, vis_params, f'Pixel Count {i+1}')

# Add a layer control panel to the map
my_map.add_child(folium.LayerControl())

# Display the map
my_map  # Or my_map.save("pixel_count_map.html") to save it

In [ ]:
import ee
import folium
import matplotlib.pyplot as plt

# Initialize Earth Engine
# auth.authenticate_user()  # Assuming you have already authenticated
ee.Initialize(project='ee-pnnwachukwu17')

# Function to aggregate pixel count
def aggregate_pixel_count(product, geometries, start_date, end_date):
    # Load MODIS ImageCollection
    collection = ee.ImageCollection(product)\
        .filterDate(start_date, end_date)

    # Calculate pixel count for each geometry
    count_images = []
    for geometry in geometries:
        count_image = collection.filterBounds(geometry) \
            .reduce(ee.Reducer.count()) \
            .select([0], ['pixel_count']) \
            .clip(geometry)
        count_images.append(count_image)

    return count_images

# Example usage:
product = 'MODIS/061/MOD13A2'  # MODIS product path
# Define rectangles for different continents (replace with your desired coordinates)
continents = [
    ee.Geometry.Rectangle([19.85,-3.16, 22.39,-1.31]),  # Africa (approx.)
    ee.Geometry.Rectangle([101.23,29.02, 102.10,29.54]),    # Asia (approx.)
    ee.Geometry.Rectangle([122.37,-33.08, 122.85,-32.49]),   # Aust_Ocean (approx.)
    ee.Geometry.Rectangle([22.45,45.12, 22.97,45.47]),  # Europe (approx.)
    ee.Geometry.Rectangle([-121.68,48.02, -119.80,49.01]),    # Nth_America (approx.)
    ee.Geometry.Rectangle([-60.70,-9.18, -58.42,-7.56]),   # Sth_America (approx.)
    # Add more rectangles for other continents
]
start_date = '2020-02-01'
end_date = '2020-04-30' # Corrected the end date to a valid date

pixel_count_images = aggregate_pixel_count(product, continents, start_date, end_date)

# Define a function to add Earth Engine layers to a folium map
def add_ee_layer(self, ee_image_object, vis_params, name, label): # Added label parameter
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True,
        show=False # Initially hide the layers
    ).add_to(self)

    # Add a label for the layer
    folium.Marker(
        location=ee_image_object.geometry().centroid().coordinates().getInfo()[::-1], # Get the center of the geometry
        popup=label,
        icon=folium.Icon(color='blue')
    ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map (adjust map center as needed)
map_center = [20, 20]  # Adjust for a global view
my_map = folium.Map(location=map_center, zoom_start=3)

# Visualization parameters
vis_params = {
    'min': 0,
    'max': 10,  # Adjust this based on expected range of pixel counts
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
}

# Labels for each continent
continent_labels = [
    'Africa', 'Asia', 'Australia/Oceania', 'Europe', 'North America', 'South America'
]

# Add the pixel count layers to the map with labels
for i, count_image in enumerate(pixel_count_images):
    my_map.add_ee_layer(count_image, vis_params, f'Pixel Count {i+1}', continent_labels[i]) # Pass the label from continent_labels

# Add a layer control panel to the map
my_map.add_child(folium.LayerControl())

# Display the map
my_map  # Or my_map.save("pixel_count_map.html") to save it

In [ ]:
import ee
import folium

# Initialize Earth Engine
# auth.authenticate_user()
ee.Initialize(project='ee-pnnwachukwu17')

def aggregate_pixel_count(product, geometry, start_date, end_date):
    collection = ee.ImageCollection(product) \
        .filterBounds(geometry) \
        .filterDate(start_date, end_date)

    count_image = collection.reduce(ee.Reducer.count()).select([0], ['pixel_count'])
    count_image = count_image.clip(geometry)
    return count_image

# Example usage:
product = 'MODIS/061/MOD13A2'
start_date = '2000-02-01'
end_date = '2024-04-30'

# Define geometries for each continent (replace with actual coordinates)
geometries = {
    'Africa': ee.Geometry.Rectangle([19.85,-3.16, 22.39,-1.31]),
    'Asia': ee.Geometry.Rectangle([101.23,29.02, 102.10,29.54]),
    'Aust_Ocean': ee.Geometry.Rectangle([122.37,-33.08, 122.85,-32.49]),
    'Europe': ee.Geometry.Rectangle([22.45,45.12, 22.97,45.47]),
    'Nth_America': ee.Geometry.Rectangle([-121.68,48.02, -119.80,49.01]),
    'Sth_America': ee.Geometry.Rectangle([-60.70,-9.18, -58.42,-7.56]),
    # ... add other continents
}

pixel_counts = {}
for continent, geometry in geometries.items():
    pixel_count_image = aggregate_pixel_count(product, geometry, start_date, end_date)
    # Get the total pixel count for the continent
    pixel_count = pixel_count_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geometry,
        scale=500,  #
        maxPixels=1e13  # Increase the maximum number of pixels allowed
        #bestEffort=True # Or set bestEffort to True to aggregate at a coarser scale
    ).get('pixel_count').getInfo()
    pixel_counts[continent] = pixel_count

# Print the pixel counts for each continent
for continent, count in pixel_counts.items():
    print(f'Pixel count for {continent}: {count}')

In [ ]:
pip install earthengine-api matplotlib geopandas folium


In [ ]:
# Import necessary libraries
import ee
import geopy
import folium
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import auth
from sklearn.metrics.pairwise import rbf_kernel
import numpy as np
from google.colab import drive
import gdown
# Import the Polygon class from shapely.geometry
from shapely.geometry import Polygon
import geopandas as gpd


In [ ]:
# Initialize Earth Engine
auth.authenticate_user()
ee.Initialize(project='ee-pnnwachukwu17')

 # For single continent like Europe

In [ ]:
# For single continent like Europe

# Initialize Earth Engine
# auth.authenticate_user()  # Assuming you have already authenticated
ee.Initialize(project='ee-pnnwachukwu17')

# Function to aggregate pixel count
def aggregate_pixel_count(product, geometry, start_date, end_date):
    # Load MODIS ImageCollection
    collection = ee.ImageCollection(product) \
        .filterBounds(geometry) \
        .filterDate(start_date, end_date)

    # Reduce the collection to get the total pixel count in the region of interest
    count_image = collection.reduce(ee.Reducer.count()).select([0], ['pixel_count']) # Select the first band and rename it

    # Clip to the geometry
    count_image = count_image.clip(geometry)

    return count_image

# Example usage:
product = 'MODIS/061/MOD13A2'  # MODIS product path
# Define a square geometry (replace with your desired coordinates)
geometry = ee.Geometry.Rectangle([
    [-122.1, 37.4],  # Bottom left corner
    [-122.0, 37.5]   # Top right corner
])
start_date = '2020-01-01'
end_date = '2020-12-31'

pixel_count_image = aggregate_pixel_count(product, geometry, start_date, end_date)

# Define a function to add Earth Engine layers to a folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map
map_center = geometry.centroid().coordinates().getInfo()[::-1]
my_map = folium.Map(location=map_center, zoom_start=12)

# Visualization parameters
vis_params = {
    'min': 0,
    'max': 10,  # Adjust this based on expected range of pixel counts
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
}

# Add the pixel count layer to the map
my_map.add_ee_layer(pixel_count_image, vis_params, 'Pixel Count')

# Add a layer control panel to the map
my_map.add_child(folium.LayerControl())

# Display the map
my_map  # Or my_map.save("pixel_count_map.html") to save it



Total number of pixels: None


In [ ]:
# This is used to print the number of pixels

# Function to aggregate pixel count
def aggregate_pixel_count(product, geometry, start_date, end_date):
    # Load MODIS ImageCollection
    collection = ee.ImageCollection(product) \
        .filterBounds(geometry) \
        .filterDate(start_date, end_date)

    # Reduce the collection to get the total pixel count in the region of interest
    count_image = collection.reduce(ee.Reducer.count()).select([0], ['pixel_count'])

    # Clip to the geometry
    count_image = count_image.clip(geometry)

    return count_image

# Example usage:
product = 'MODIS/061/MOD13A2'  # MODIS product path

# Define a square geometry (replace with your desired coordinates)
geometry = ee.Geometry.Rectangle([
    22.45,45.12,  # Lower left corner (longitude, latitude)
    22.97,45.47   # Upper right corner (longitude, latitude)
])

start_date = '2020-01-01'
end_date = '2020-12-31'

pixel_count_image = aggregate_pixel_count(product, geometry, start_date, end_date)

# Get the total number of pixels
pixel_count = pixel_count_image.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=geometry,
    scale=500  # Adjust scale as needed
).get('pixel_count').getInfo()

print("Total number of pixels:", pixel_count)

# ... (rest of the code for creating and displaying the map)

Total number of pixels: 207374.13333333423


 # For Multiple Continents

In [ ]:
import ee
import folium
import matplotlib.pyplot as plt

# Initialize Earth Engine
# auth.authenticate_user()  # Assuming you have already authenticated
ee.Initialize(project='ee-pnnwachukwu17')

# Function to aggregate pixel count
def aggregate_pixel_count(product, geometries, start_date, end_date):
    # Load MODIS ImageCollection
    collection = ee.ImageCollection(product)\
        .filterDate(start_date, end_date)

    # Calculate pixel count for each geometry
    count_images = []
    for geometry in geometries:
        count_image = collection.filterBounds(geometry) \
            .reduce(ee.Reducer.count()) \
            .select([0], ['pixel_count']) \
            .clip(geometry)
        count_images.append(count_image)

    return count_images

# Example usage:
product = 'MODIS/061/MOD13A2'  # MODIS product path
# Define rectangles for different continents (replace with your desired coordinates)
continents = [
    ee.Geometry.Rectangle([19.85,-3.16, 22.39,-1.31]),  # Africa (approx.)
    ee.Geometry.Rectangle([101.23,29.02, 102.10,29.54]),    # Asia (approx.)
    ee.Geometry.Rectangle([122.37,-33.08, 122.85,-32.49]),   # Aust_Ocean (approx.)
    ee.Geometry.Rectangle([22.45,45.12, 22.97,45.47]),  # Europe (approx.)
    ee.Geometry.Rectangle([-121.68,48.02, -119.80,49.01]),    # Nth_America (approx.)
    ee.Geometry.Rectangle([-60.70,-9.18, -58.42,-7.56]),   # Sth_America (approx.)
    # Add more rectangles for other continents
]
start_date = '2020-01-01'
end_date = '2020-12-30'

pixel_count_images = aggregate_pixel_count(product, continents, start_date, end_date)

# Define a function to add Earth Engine layers to a folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map (adjust map center as needed)
map_center = [20, 20]  # Adjust for a global view
my_map = folium.Map(location=map_center, zoom_start=3)

# Visualization parameters
vis_params = {
    'min': 0,
    'max': 10,  # Adjust this based on expected range of pixel counts
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
}

# Add the pixel count layers to the map
for i, count_image in enumerate(pixel_count_images):
    my_map.add_ee_layer(count_image, vis_params, f'Pixel Count {i+1}')

# Add a layer control panel to the map
my_map.add_child(folium.LayerControl())

# Display the map
my_map  # Or my_map.save("pixel_count_map.html") to save it

In [ ]:
import ee
import folium
import matplotlib.pyplot as plt

# Initialize Earth Engine
# auth.authenticate_user()  # Assuming you have already authenticated
ee.Initialize(project='ee-pnnwachukwu17')

# Function to aggregate pixel count
def aggregate_pixel_count(product, geometries, start_date, end_date):
    # Load MODIS ImageCollection
    collection = ee.ImageCollection(product)\
        .filterDate(start_date, end_date)

    # Calculate pixel count for each geometry
    count_images = []
    for geometry in geometries:
        count_image = collection.filterBounds(geometry) \
            .reduce(ee.Reducer.count()) \
            .select([0], ['pixel_count']) \
            .clip(geometry)
        count_images.append(count_image)

    return count_images

# Example usage:
product = 'MODIS/061/MOD13A2'  # MODIS product path
# Define rectangles for different continents (replace with your desired coordinates)
continents = [
    ee.Geometry.Rectangle([19.85,-3.16, 22.39,-1.31]),  # Africa (approx.)
    ee.Geometry.Rectangle([101.23,29.02, 102.10,29.54]),    # Asia (approx.)
    ee.Geometry.Rectangle([122.37,-33.08, 122.85,-32.49]),   # Aust_Ocean (approx.)
    ee.Geometry.Rectangle([22.45,45.12, 22.97,45.47]),  # Europe (approx.)
    ee.Geometry.Rectangle([-121.68,48.02, -119.80,49.01]),    # Nth_America (approx.)
    ee.Geometry.Rectangle([-60.70,-9.18, -58.42,-7.56]),   # Sth_America (approx.)
    # Add more rectangles for other continents
]
start_date = '2020-02-01'
end_date = '2020-04-30' # Corrected the end date to a valid date

pixel_count_images = aggregate_pixel_count(product, continents, start_date, end_date)

# Define a function to add Earth Engine layers to a folium map
def add_ee_layer(self, ee_image_object, vis_params, name, label): # Added label parameter
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True,
        show=False # Initially hide the layers
    ).add_to(self)

    # Add a label for the layer
    folium.Marker(
        location=ee_image_object.geometry().centroid().coordinates().getInfo()[::-1], # Get the center of the geometry
        popup=label,
        icon=folium.Icon(color='blue')
    ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map (adjust map center as needed)
map_center = [20, 20]  # Adjust for a global view
my_map = folium.Map(location=map_center, zoom_start=3)

# Visualization parameters
vis_params = {
    'min': 0,
    'max': 10,  # Adjust this based on expected range of pixel counts
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
}

# Labels for each continent
continent_labels = [
    'Africa', 'Asia', 'Australia/Oceania', 'Europe', 'North America', 'South America'
]

# Add the pixel count layers to the map with labels
for i, count_image in enumerate(pixel_count_images):
    my_map.add_ee_layer(count_image, vis_params, f'Pixel Count {i+1}', continent_labels[i]) # Pass the label from continent_labels

# Add a layer control panel to the map
my_map.add_child(folium.LayerControl())

# Display the map
my_map  # Or my_map.save("pixel_count_map.html") to save it

In [ ]:
import ee
import folium

# Initialize Earth Engine
# auth.authenticate_user()
ee.Initialize(project='ee-pnnwachukwu17')

def aggregate_pixel_count(product, geometry, start_date, end_date):
    collection = ee.ImageCollection(product) \
        .filterBounds(geometry) \
        .filterDate(start_date, end_date)

    count_image = collection.reduce(ee.Reducer.count()).select([0], ['pixel_count'])
    count_image = count_image.clip(geometry)
    return count_image

# Example usage:
product = 'MODIS/061/MOD13A2'
start_date = '2000-02-01'
end_date = '2024-04-30'

# Define geometries for each continent (replace with actual coordinates)
geometries = {
    'Africa': ee.Geometry.Rectangle([19.85,-3.16, 22.39,-1.31]),
    'Asia': ee.Geometry.Rectangle([101.23,29.02, 102.10,29.54]),
    'Aust_Ocean': ee.Geometry.Rectangle([122.37,-33.08, 122.85,-32.49]),
    'Europe': ee.Geometry.Rectangle([22.45,45.12, 22.97,45.47]),
    'Nth_America': ee.Geometry.Rectangle([-121.68,48.02, -119.80,49.01]),
    'Sth_America': ee.Geometry.Rectangle([-60.70,-9.18, -58.42,-7.56]),
    # ... add other continents
}

pixel_counts = {}
for continent, geometry in geometries.items():
    pixel_count_image = aggregate_pixel_count(product, geometry, start_date, end_date)
    # Get the total pixel count for the continent
    pixel_count = pixel_count_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geometry,
        scale=500,  # Adjust scale as needed
        maxPixels=1e13  # Increase the maximum number of pixels allowed
        #bestEffort=True # Or set bestEffort to True to aggregate at a coarser scale
    ).get('pixel_count').getInfo()
    pixel_counts[continent] = pixel_count

# Print the pixel counts for each continent
for continent, count in pixel_counts.items():
    print(f'Pixel count for {continent}: {count}')

Pixel count for Africa: 129715142.91764683
Pixel count for Asia: 12479566.815686226
Pixel count for Aust_Ocean: 7814286.243137279
Pixel count for Europe: 5020308.470588274
Pixel count for Nth_America: 51338842.070588194
Pixel count for Sth_America: 101914020.35686333
